In [ ]:
# bill management system

# Connect to the database
import sqlite3
conn = sqlite3.connect("bills.db")
cur = conn.cursor()

# Create tables
cur.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    phone TEXT
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS bills (
    bill_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
)
''')

cur.execute('''
CREATE TABLE IF NOT EXISTS items (
    item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    bill_id INTEGER,
    item_name TEXT,
    quantity INTEGER,
    price REAL,
    FOREIGN KEY(bill_id) REFERENCES bills(bill_id)
)
''')

conn.commit()

# Function to create a new bill
def create_bill(name, phone):
    cur.execute("INSERT INTO customers (name, phone) VALUES (?, ?)", (name, phone))
    customer_id = cur.lastrowid
    cur.execute("INSERT INTO bills (customer_id) VALUES (?)", (customer_id,))
    bill_id = cur.lastrowid
    conn.commit()
    print(f" Bill created successfully for {name}. Bill ID: {bill_id}")
    return bill_id

# Function to add an item to a bill
def add_item_to_bill(bill_id, item_name, quantity, price):
    cur.execute("INSERT INTO items (bill_id, item_name, quantity, price) VALUES (?, ?, ?, ?)",
                (bill_id, item_name, quantity, price))
    conn.commit()
    print(f" Item '{item_name}' added to Bill ID: {bill_id}")

# Function to view a bill
def view_bill(bill_id):
    cur.execute('''
        SELECT c.name, c.phone, b.date
        FROM bills b
        JOIN customers c ON b.customer_id = c.customer_id
        WHERE b.bill_id = ?
    ''', (bill_id,))
    bill_info = cur.fetchone()

    if not bill_info:
        print(" Bill not found.")
        return

    print(f"\n--- Bill ID: {bill_id} ---")
    print(f"Customer: {bill_info[0]}")
    print(f"Phone: {bill_info[1]}")
    print(f"Date: {bill_info[2]}")
    print("\nItems:")
    print("{} {} {} {}".format("Item", "Qty", "Price", "Total"))

    cur.execute('''
        SELECT item_name, quantity, price FROM items WHERE bill_id = ?
    ''', (bill_id,))
    items = cur.fetchall()

    grand_total = 0
    for item in items:
        name, qty, price = item
        total = qty * price
        grand_total += total
        print("{} {} {} {}".format(name, qty, price, total))

    print(f"\nTotal Amount: ₹{grand_total}")

# Function to view items in a bill with item_id
def view_items_in_bill(bill_id):
    cur.execute('''
        SELECT item_id, item_name, quantity, price FROM items WHERE bill_id = ?
    ''', (bill_id,))
    items = cur.fetchall()

    if not items:
        print(" No items found in this bill.")
        return []

    print("\nItems in Bill:")
    print("{} {} {} {}".format("Item ID", "Item Name", "Qty", "Price"))
    for item in items:
        item_id, name, qty, price = item
        print("{} {} {} {}".format(item_id, name, qty, price))
    return items

# Function to delete an item from bill
def delete_item_from_bill():
    try:
        bill_id = int(input("Enter Bill ID: "))
        items = view_items_in_bill(bill_id)
        if not items:
            return

        item_id = int(input("Enter Item ID to delete: "))
        cur.execute("DELETE FROM items WHERE item_id = ? AND bill_id = ?", (item_id, bill_id))
        conn.commit()
        print(f" Item ID {item_id} deleted successfully from Bill ID {bill_id}.")
    except Exception as e:
        print(" Error deleting item:", e)

# Main menu
def bill_menu():
    while True:
        print("\n===== Bill Management System =====")
        print("1. Create New Bill")
        print("2. Add Item to Bill")
        print("3. View Bill")
        print("4. Delete Item from Bill")
        print("5. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            name = input("Enter customer name: ")
            phone = input("Enter phone number: ")
            create_bill(name, phone)
        elif choice == '2':
            bill_id = int(input("Enter Bill ID: "))
            item_name = input("Enter item name: ")
            qty = int(input("Enter quantity: "))
            price = float(input("Enter price per unit: ₹"))
            add_item_to_bill(bill_id, item_name, qty, price)
        elif choice == '3':
            bill_id = int(input("Enter Bill ID to view: "))
            view_bill(bill_id)
        elif choice == '4':
            delete_item_from_bill()
        elif choice == '5':
            print("Exiting Bill Management System. Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

bill_menu()
conn.close()


===== Bill Management System =====
1. Create New Bill
2. Add Item to Bill
3. View Bill
4. Delete Item from Bill
5. Exit


Enter your choice:  1
Enter customer name:  Pawan
Enter phone number:  9823934232


 Bill created successfully for Pawan. Bill ID: 1

===== Bill Management System =====
1. Create New Bill
2. Add Item to Bill
3. View Bill
4. Delete Item from Bill
5. Exit


Enter your choice:  2
Enter Bill ID:  1
Enter item name:  Pen
Enter quantity:  10
Enter price per unit: ₹ 10


 Item 'Pen' added to Bill ID: 1

===== Bill Management System =====
1. Create New Bill
2. Add Item to Bill
3. View Bill
4. Delete Item from Bill
5. Exit


Enter your choice:  3
